# Abstract

Airbnb is an online community marketplace for people to book a complete travel experience, including accommodations and locally curated activities. One of the first interactions a guest has with a home is the listing description. The listing refers to the description of the home, and provides an opportunity to highlight unique features. As it turns out, the listing description occupies a decisive role in how guests perceive the overall quality of a home. It is so important that homes qualifying for the Airbnb Plus program (a collection of homes verified for quality, comfort and style) have their listing descriptions written by professional writers trained in the Airbnb Plus editorial style. But writing a unique and captivating listing description, that stands out from other homes can be difficult. Airbnb offers resources for writing a great listing description on their [blog](https://blog.atairbnb.com/writing-great-listing-descriptions/), and also includes [tips](https://www.airbnb.com/help/article/431/how-do-i-make-my-listing-more-competitive) on how to make a listing competitive. 

However, this can be difficult to implement in practice for many hosts, who have no way of receiving feedback until after the guest leaves a rating. To help hosts maximize the probability of earning a high rating, I propose a text classification system that will allow hosts to receive instant feedback on their listing descriptions. The classifier is built on top of Google's state-of-the-art pre-trained model, BERT, a bidirectional transformer model. BERT is powerful because it understands the contextual information to the left and right of a word, giving it a deeper understanding of the language model. BERTs outputs were fed into a deep feedforward neural network with a dropout layer (to enforce regularization) and a softmax classifier. Trained on roughly 7,000 listing descriptions in San Francisco, the model achieved 81.3% accuracy in classifying the star rating a host will receive based on their listing description. In addition, the data showed that guests value private homes above corporate properties, and prefer listing descriptions that emphasize the unique qualities of a home rather than a list of generic amenities. Accordingly, hosts are recommended to focus on these unique characteristics in their descriptions, and include words that evoke homey feelings.

# 1 Obtain

The data is available at [Inside Airbnb](http://insideairbnb.com/san-francisco/#), a site that collects detailed information from some of the major cities Airbnb operates in. For this project, the data was downloaded from the San Francisco dataset as a series of csv files. 

# 2 Scrub

Preprocess the data for exploratory analysis, which primarily entails handling missing values by discretization and replacing null values.

In [1]:
# import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

%matplotlib inline
plt.style.use('seaborn')

warnings.filterwarnings(action='ignore')

In [2]:
# load data
listing = pd.read_csv('listing.csv')
listing.head()

,description,host_is_superhost,latitude,longitude,price,number_of_reviews,review_scores_rating
0,New update: the house next door is under const...,t,37.76931,-122.43386,170,180,97.0
1,We live in a large Victorian house on a quiet ...,f,37.74511,-122.42102,235,111,98.0
2,Nice and good public transportation. 7 minute...,f,37.76669,-122.45250,65,17,85.0
3,Nice and good public transportation. 7 minute...,f,37.76487,-122.45183,65,8,93.0
4,Pls email before booking. Interior featured i...,f,37.77525,-122.43637,785,27,97.0


In [3]:
# convert review_scores_rating to a 5 point scale
listing['review_scores_rating'] = listing['review_scores_rating']/20

# discretize review_scores_rating
listing['review_scores_rating'] = pd.cut(listing['review_scores_rating'], bins=5,
                                         labels=['0.0-1.0', '1.0-2.0', '2.0-3.0', '3.0-4.0', '4.0-5.0'])

# replace null values in description and review_scores_rating
listing['description'].replace(np.NaN, 'no_description', inplace=True)
listing['review_scores_rating'].replace(np.NaN, 'no_review', inplace=True)

* **review_scores_rating** is scored on a 100-point scale. To keep the data in line with what hosts generally see, convert this value to a 5-point scale, the star rating that hosts are accustomed to. The **review_scores_rating** is equivalent to the **overall star rating**, an aggregated score for each category a guest can give a rating on. 


* Discretize **review_scores_rating** to handle any missing data by replacing with **no_review**. We can't impute these missing values since they are intentionally missing because the guest did not leave a review.


* Likewise, empty fields in **description** have been deliberately left blank by the host and can not be imputed.

# 3 Explore

### 3.1 Geospatial analysis of star ratings and price

[kepler.gl](http://kepler.gl/) is a geospatial analysis tool developed by Uber's Data Vis team. It functions best with large-scale datasets, but also works beautifully with smaller ones. Simply drag and drop a dataset into the web-based app, and then begin applying filters, layers and colors to bring geospatial data to life. 

Set the color in the hexbin price layer by the **median** price instead of the **average** to avoid misrepresenting the data. Since the average is more sensitive to extreme values, we want to avoid artificially inflating the price of an area. Polar price points in a given area, are not uncommon in San Francisco where wealthy neighborhoods are often bordered by less affluent ones.

![keplerPlot](imgs/kepler-gl.png)
**Fig. 1** A layered hexbin and point map of ratings and price of Airbnb listings in San Francisco.

### 3.2 Characteristic terms and associations

Currently, there aren't very many good options for text visualization. Word clouds are perhaps the most common method, but they can be hard to interpret. It can be difficult to compare the sizes of two non-horizontally adjacent words, and longer words can appear to have an outsized impact simply because they occupy more space. 

To avoid these pitfalls, [Scattertext](https://github.com/JasonKessler/scattertext) is a visualization tool that presents a scatterplot, where each axis corresponds to the frequency of a term and its related category. The term associations are determined by their scaled F-score, the harmonic mean of precision and frequency. The scaled F-score takes the normal cumulative density function of the precision and frequency scores, to scale and standardize both scores. Terms that are associated will have both high category precision and frequency, producing a higher scaled F-score.

#### Plot guide
* Terms used commonly in all listing descriptions are located in the upper-right corner of the plot (home, house, kitchen, apartment, room). These words are all common elements of a living space regardless of high or low star rating. On the other hand, terms used infrequently in all listing descriptions are located in the lower-left corner of the plot. For example, children is rarely used, which suggests homes accommodating children are relatively rare. 


* It gets more interesting when we look at the upper-left and bottom-right corners. These are terms that are most commonly associated with high star ratings and all other ratings respectively. The colors also help identify word association. Those terms that are more aligned with high star ratings are blue, and those more associated with all other ratings are red.  


* Scattertext is an [interactive plot](https://textexplorer.herokuapp.com/), and clicking on a word brings up an excerpt from where it was used in the listing description. There is also a search function if you want to look up a specific word. Finally, Scattertext is unable to compare multiple groups at once, which is why we need to take a one-to-many visualization approach.

In [4]:
# import libraries for handling text
import scattertext as st
import spacy
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from pprint import pprint

# load English model
nlp = spacy.load('en')

# create a scattertext corpus & look for differences between listing descriptions in ratings
# remove stop words
corpus = (st.CorpusFromPandas(listing,
                              category_col='review_scores_rating',
                              text_col='description',
                              nlp=nlp)
          .build()
          .remove_terms(ENGLISH_STOP_WORDS, ignore_absences=True))

# generate scatterplot and save to HTML file
# set a minimum term frequency to filter out infrequent words
html = st.produce_scattertext_explorer(corpus,
          category='4.0-5.0',
          category_name='Star Rating 4.0 - 5.0',
          not_category_name='All Other Ratings',
          width_in_pixels=1000,
          minimum_term_frequency=200)
open('term-associations.html', 'wb').write(html.encode('utf-8'))

6852412

![termAssociations](imgs/termAssociations.png)
**Fig. 2** Scatterplot of terms associated with high star ratings and all other ratings.

### 3.3 Topic signals and categories

It is helpful to understand what types of topics are being mentioned in listing descriptions that correspond with high star ratings. Hosts can model and incorporate these topic categories into their own listing descriptions, while learning which ones to avoid.

[Empath](https://hci.stanford.edu/publications/2016/ethan/empath-chi-2016.pdf) allows us to learn the topic signals within listing descriptions, by generating and validating categories from a few seed terms. For example, the terms vintage, trendy and fashion would fall under the hipster category in Empath. Empath works by learning the word representations from 1.8 billion words from fiction stories with a neural network, and creates a vector space that measures the cosine similarity between words. Given a set of new seed terms, Empath utilizes its learned neural weights to discover related terms, and then generates a new lexical category.

Integrating Empath with Scattertext, we can build a scatterplot of topics associated with high star ratings and all other ratings. 

In [5]:
# initialize Empath topics/categories
feat_builder = st.FeatsFromOnlyEmpath()

# create corpus from Empath
empath_corpus = (st.CorpusFromParsedDocuments(listing,
                                             category_col='review_scores_rating',
                                             feats_from_spacy_doc=feat_builder,
                                             parsed_col='description')
                .build())

# generate scatterplot and save to HTML file
# set use_non_text_features=True to use labeled Empath topics/categories
# set topic_model_term_lists=feat_builder.get_top_model_term_lists to ensure relevant terms are bolded in excerpts
html = st.produce_scattertext_explorer(empath_corpus,
                                       category='4.0-5.0',
                                       category_name='Star Rating 4.0-5.0',
                                       not_category_name='All Other Ratings',
                                       width_in_pixels=1000,
                                       use_non_text_features=True,
                                       use_full_doc=True,
                                       topic_model_term_lists=feat_builder.get_top_model_term_lists())
open('term-Empath.html', 'wb').write(html.encode('utf-8'))

10617716

![termEmpath](imgs/term-Empath.png)
**Fig. 3** A scatterplot of lexical categories associated with high star ratings and all other ratings.

### 3.4 Word similarity

It may also be interesting to create a plot of word similarities to visualize how listing descriptions with high star ratings differ in words used. A word2vec model was used to generate word embeddings of the listing descriptions, and then tSNE (t-distributed stochastic neighbor embedding) was used to reduce dimensionality.

A word2vec model is simply a vector representation of a word, and maps words to a vector of real numbers (otherwise known as a word embedding). Two common word embedding models are the continuous bag of words model and the skip-gram model. The latter assumes that context words are generated from a central target word, while the former assumes the opposite.

Generating word embeddings can create highly dimensional data, which is why its useful to employ a dimensionality reduction technique like tSNE. tSNE is an implementation of manifold learning, a technique for learning non-linear structure in data. The advantages of tSNE include reducing tendency of crowding points around the center, and exposing natural clusters within the data. Read [here](https://lvdmaaten.github.io/tsne/) for more details about tSNE.

In [6]:
# import necessary libraries
from sklearn.manifold import TSNE
from gensim.models.word2vec import Word2Vec

# split sentences in listing description with st.whitespace_nlp_with_sentences
listing['parse'] = listing['description'].apply(st.whitespace_nlp_with_sentences)

# create a stop-listed corpus of unigram terms
corpus = (st.CorpusFromParsedDocuments(listing, 
                                       category_col='review_scores_rating', 
                                       parsed_col='parse')
          .build()
          .get_stoplisted_unigram_corpus())

# generate scatterplot and save to HTML file
# learn word embeddings with a word2vec model (specify more worker threads for faster processing)
# use t-SNE projection model to visualize word similarity in two-dimensions
html = st.produce_projection_explorer(corpus,
                                      word2vec_model=Word2Vec(workers=4),
                                      projection_model=TSNE(),
                                      category='4.0-5.0',
                                      category_name='Star Rating 4.0-5.0',
                                      not_category_name='All Other Ratings')  
open('wordSimilarity.html', 'wb').write(html.encode('utf-8'))

8111972

![tSNE](imgs/tSNE.png)
**Fig. 4** Word similarity clusters are revealed with tSNE.

# 4 Model

Bidirectional Encoder Representations from Transformers ([BERT](https://github.com/google-research/bert)) is a state-of-the-art model released by Google in late 2018. It is a method for adapting pre-trained language representations for downstream NLP tasks. What makes BERT truly unique is its bidirectionality, meaning, it captures contextual information to the left and right of a word. Training bidirectional models are inherently difficult though because it would allow the word that's being predicted to indirectly see itself in a multi-layered context. This hurdle is overcome by randomly masking some words, and then asking BERT to predict the masked word. To train BERT to understand relationships between two sentences, during the pre-training process, BERT is further tasked with predicting whether one sentence follows another.

There are two stages in using BERT: pre-training and fine-tuning.

* **Pre-training** is the process of training the BERT model on an extremely large text corpus (Wikipedia) to learn language representations. This is a highly expensive task (four days of training on 4-16 cloud TPUs!), but thankfully, the researchers at Google have already completed this step and have publicly released their pre-trained models. All we have to do is download these pre-trained models. For this project, we will use the Uncased BERT-Base model, which simply means all the text has been lowercased, and accent markers have been stripped. The BERT-Base model architecture contains 12 Transformer blocks, 768 hidden units, and 12 attention heads. 

* **Fine-tuning** is the stage where we adapt the BERT model to our classification task. On a CPU, this process took 14 hrs, so we recommend using Google's Colaboratory notebook, and accelerate the process by taking advantage of their cloud GPUs.

The following code is adapted from Google's [BERT FineTuning with Cloud TPU](https://colab.research.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb) notebook. 

### 4.1 Preprocess the data

It is common in NLP to tokenize text such that each word is a separate entity. BERT uses [WordPiece model](https://arxiv.org/pdf/1609.08144.pdf) tokenization, where each word is further segmented into sub-word units. A special **[CLS]** (classifier) token is appended to the beginning of the text, and a **[SEP]** (separator) token is inserted at the end. The WordPiece tokens are generated by optimizing the number of tokens, such that the resulting corpus contains the minimal number of WordPieces when segmented according to the model. By creating sub-word units, WordPiece allows BERT to capture out-of-vocabulary words and store only 30,522 words, which effectively gives it an evolving word dictionary. 

In summary, WordPiece tokenization achieves a good balance between vocabulary size and rare words. 

In [7]:
# install BERT
!pip install bert-tensorflow

# import necessary libraries for BERT
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_hub as hub
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

# set output directory to store model output and checkpoints
output_dir = 'modelOutput'

W0325 21:19:27.208451 4584494528 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [8]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# encode categorical variables
listing['review_scores_rating'] = LabelEncoder().fit_transform(listing['review_scores_rating'])

# create training and test sets
train, test = train_test_split(listing, test_size=0.25)

# create feature and target variables
# create a list of labels
DATA_COLUMN = 'description'
LABEL_COLUMN = 'review_scores_rating'
label_list = [0, 1, 2, 3, 4, 5]

In [9]:
# preprocess data into format BERT understands
train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None,
                                                                             text_a=x[DATA_COLUMN],
                                                                             text_b=None,
                                                                             label=x[LABEL_COLUMN]), axis = 1)
test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None,
                                                                           text_a=x[DATA_COLUMN],
                                                                           text_b=None,
                                                                           label=x[LABEL_COLUMN]), axis = 1)

In [10]:
# create a tokenizer from uncased BERT model
BERT_MODEL_HUB = 'https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1'

def create_tokenizer_from_hub():
    with tf.Graph().as_default():
        bert_module=hub.Module(BERT_MODEL_HUB)
        tokenization_info=bert_module(signature='tokenization_info', as_dict=True)
        with tf.Session() as sess:
            vocab_file, do_lower_case = sess.run([tokenization_info['vocab_file'],
                                                  tokenization_info['do_lower_case']])
    return bert.tokenization.FullTokenizer(
    vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0325 21:19:56.035194 4584494528 tf_logging.py:115] Saver not created because there are no variables in the graph to restore


In [11]:
# set max sequence length
MAX_SEQ_LENGTH = 256

# convert features to InputFeatures that BERT understands
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, 
                                                                  label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, 
                                                                  label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 5363


I0325 21:19:59.225611 4584494528 tf_logging.py:115] Writing example 0 of 5363


INFO:tensorflow:*** Example ***


I0325 21:19:59.232403 4584494528 tf_logging.py:115] *** Example ***


INFO:tensorflow:guid: None


...

I0325 21:20:15.819514 4584494528 tf_logging.py:115] input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


I0325 21:20:15.821882 4584494528 tf_logging.py:115] segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 4 (id = 4)


I0325 21:20:15.825112 4584494528 tf_logging.py:115] label: 4 (id = 4)


### 4.2 Create the model

In [12]:
# create function for the model
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels, num_labels):
    
    bert_module = hub.Module(
        BERT_MODEL_HUB,
        trainable=True)
    bert_inputs = dict(
        input_ids=input_ids,
        input_mask=input_mask,
        segment_ids=segment_ids)
    bert_outputs = bert_module(
        inputs=bert_inputs,
        signature='tokens',
        as_dict=True)
    
    # use pooled_output for classification tasks
    output_layer = bert_outputs['pooled_output']
    hidden_size = output_layer.shape[-1].value
    
    output_weights = tf.get_variable(
        'output_weights', [num_labels, hidden_size],
        initializer=tf.truncated_normal_initializer(stddev=0.02))
    
    output_bias = tf.get_variable(
        'output_bias', [num_labels], initializer=tf.zeros_initializer())
    
    with tf.variable_scope('loss'):
        
        # add dropout layer to prevent overfitting
        output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)
        logits = tf.matmul(output_layer, output_weights, transpose_b=True)
        logits = tf.nn.bias_add(logits, output_bias)
        log_probs = tf.nn.log_softmax(logits, axis=-1)
        
        # one-hot encode labels
        one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)
        
        predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
        # obtain predicted labels and probabilities
        if is_predicting:
            return (predicted_labels, log_probs)
        
        # compute loss between actual and predicted labels during evaluation
        per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
        loss = tf.reduce_mean(per_example_loss)
        return (loss, predicted_labels, log_probs)

In [18]:
# wrap model function in model_fn_builder that allows model to train, evaluate and predict
def model_fn_builder(num_labels, learning_rate, num_train_steps, num_warmup_steps):
    
    def model_fn(features, labels, mode, params):
        
        input_ids = features['input_ids']
        input_mask = features['input_mask']
        segment_ids = features['segment_ids']
        label_ids = features['label_ids']
        
        is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
        
        if not is_predicting:
            (loss, predicted_labels, log_probs) = create_model(
                is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)
            
            train_op = bert.optimization.create_optimizer(
                loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)
            
            # calculate evaluation metrics
            def metric_fn(label_ids, predicted_labels):
                accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
                return {
                    "eval_accuracy": accuracy,
                }
            
            eval_metrics = metric_fn(label_ids, predicted_labels)
            
            if mode == tf.estimator.ModeKeys.TRAIN:
                return tf.estimator.EstimatorSpec(mode=mode,
                    loss=loss,
                    train_op=train_op)
            else:
                return tf.estimator.EstimatorSpec(mode=mode,
                    loss=loss,
                    eval_metric_ops=eval_metrics)
                
        else:
            (predicted_labels, log_probs) = create_model(
                is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)
            predictions = {
                'probabilities': log_probs,
                'labels': predicted_labels
            }
            return tf.estimator.EstimatorSpec(mode, predictions=predictions)

    return model_fn

In [19]:
# set hyperparameters
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
WARMUP_PROPORTION = 0.1
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

In [20]:
# compute number of training and warmup steps
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [21]:
# specify output directory and checkpoints to save
run_config = tf.estimator.RunConfig(
    model_dir='modelOutput',
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [22]:
# build estimator
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})

INFO:tensorflow:Using config: {'_model_dir': 'learn-co/capstone', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a31c5bba8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I0325 21:21:15.399399 4584494528 tf_logging.py:115] Using config: {'_model_dir': 'learn-co/capstone', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a31c5bba8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [23]:
# create input function for training
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

In [30]:
# begin training
print(f'Beginning training')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("training took time ", datetime.now() - current_time)

Beginning training
INFO:tensorflow:Calling model_fn.


I0325 21:22:33.721390 4584494528 tf_logging.py:115] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0325 21:22:37.773957 4584494528 tf_logging.py:115] Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving checkpoints for 502 into learn-co/capstone/model.ckpt.


I0326 11:22:18.215119 4584494528 tf_logging.py:115] Saving checkpoints for 502 into learn-co/capstone/model.ckpt.


INFO:tensorflow:Loss for final step: 0.28876072.


I0326 11:22:28.397181 4584494528 tf_logging.py:115] Loss for final step: 0.28876072.


training took time  14:00:00.211425


### 4.3 Evaluate model on test data

In [32]:
# create input function for evaluation
test_input_fn = run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

# get evaluation metrics
estimator.evaluate(input_fn=test_input_fn, steps=None)

INFO:tensorflow:Calling model_fn.


I0326 11:26:43.519441 4584494528 tf_logging.py:115] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0326 11:26:46.432037 4584494528 tf_logging.py:115] Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving dict for global step 502: eval_accuracy = 0.81263983, global_step = 502, loss = 0.57338


I0326 11:54:22.891098 4584494528 tf_logging.py:115] Saving dict for global step 502: eval_accuracy = 0.81263983, global_step = 502, loss = 0.57338


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 502: learn-co/capstone/model.ckpt-502


I0326 11:54:25.874749 4584494528 tf_logging.py:115] Saving 'checkpoint_path' summary for global step 502: learn-co/capstone/model.ckpt-502


{'eval_accuracy': 0.81263983, 'loss': 0.57338, 'global_step': 502}

In [39]:
# create function to take new predictions
def getPrediction(text):
    labels = ['no_review', '0.0-1.0', '1.0-2.0', '2.0-3.0', '3.0-4.0', '4.0-5.0']
    input_examples = [run_classifier.InputExample(guid=None, text_a = x, text_b = None, label = 0) for x in text]
    input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
    predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
    predictions = estimator.predict(predict_input_fn)
    return [(sentence, prediction['probabilities'], labels[prediction['labels']]) for sentence, prediction in zip(text, predictions)]

In [44]:
# insert text from listing description
text = ['''We live in an 1895 Victorian top flat with 12 foot ceilings on the best block in San Francisco. 
           Our calendar is always up to date. Location Location Location~ Our home is close to the Mission, 
           Lower Haight, and 2.5 blocks from the Castro theater. The underground is 3 blocks away, with a 
           street car even closer. The room is small and cosy, but a great price for one of the most 
           expensive neighborhoods in the US!''',
    '']

In [45]:
# save results to predictions
predictions = getPrediction(text)

INFO:tensorflow:Writing example 0 of 2


I0326 13:14:28.473710 4584494528 tf_logging.py:115] Writing example 0 of 2


INFO:tensorflow:*** Example ***


I0326 13:14:28.482618 4584494528 tf_logging.py:115] *** Example ***


INFO:tensorflow:guid: None


...

I0326 13:14:34.690333 4584494528 tf_logging.py:115] Restoring parameters from learn-co/capstone/model.ckpt-502


INFO:tensorflow:Running local_init_op.


I0326 13:14:37.429188 4584494528 tf_logging.py:115] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0326 13:14:37.549880 4584494528 tf_logging.py:115] Done running local_init_op.


In [46]:
predictions[0]

('We live in an 1895 Victorian top flat with 12 foot ceilings on the best block in San Francisco. \n           Our calendar is always up to date. Location Location Location~ Our home is close to the Mission, \n           Lower Haight, and 2.5 blocks from the Castro theater. The underground is 3 blocks away, with a \n           street car even closer. The room is small and cosy, but a great price for one of the most \n           expensive neighborhoods in the US!',
 array([-6.959856 , -6.916402 , -6.4755964, -4.4082885, -0.0322696,
        -4.1291037], dtype=float32),
 '3.0-4.0')

# 5 Interpret

## 5.1 Geospatial analysis

**Supply**: Figure 1 highlights a geographically inbalanced supply of homes which is a combined result of:

   1. **Tourist attractions**: There is a greater demand for homes on the eastern side of the city because it is closer to tourist attractions, downtown and overall commercial activity. As we move over to West San Francisco, it becomes more residential with fewer attractions. Homes in West and South San Francisco are set at a lower price as a result. An exception to this general trend is the Tenderloin, a high-crime neighborhood, which acts to drive down price.
    
   2. **Lifestyle disparities**: It is also possible that there is a lower supply of homes in West San Francisco due to lifestyle disparities. In West San Francisco, homes are primarily occupied by families in contrast to the young professionals in East San Francisco. The latter group is more open to renting out any free space they have, while families will want to maintain their privacy. As a result, there are fewer available spaces in West San Francisco.
    
   3. **Age gap**: Related to lifestyle disparities is age gap, which may play a role in the adoption of Airbnb. As mentioned previously, West San Francisco is a mostly residential neighborhood with an older population. These individuals may be more reluctant to rent out their homes to strangers, and slow to embrace the idea of home-sharing. 
    

**Price**: The impact of the inbalanced supply of homes is reflected in the price. Homes are more expensive in Northeast San Francisco due to the combined influences of the above factors. 


**Rating**: The effect of rating on price is most clearly demonstrated in South San Francisco (Ocean View, Portola, Bayview), where lower ratings coincide with low prices. This effect is mitigated somewhat in the Tenderloin, due to better ratings in the surrounding favorable neighborhoods.

## 5.2 Term associations

In Figure 2, one of the more interesting points is how geographical information is captured in the scatterplot. Neighborhoods like Bernal Heights/Castro are more commonly associated with high star ratings, while Cole Valley/Chinatown are associated with all other ratings. Notice how Bernal Heights/Castro have a higher median price than Cole Valley/Chinatown, further reinforcing the notion that price and star rating are related.


The most differentiating words are private homes and properties owned by home-share companies (Sonder & Zeus). Private homes are more commonly associated with a high star rating, while Sonder & Zeus are more frequently used in all other ratings. This suggests people may have a better experience with private homes, because it feels more organic/natural. It is more in line with Airbnb's vision of co-inhabiting another individual's home, and immersing yourself in their culture, rather than a company-owned property. 


Listing descriptions with high star ratings also tend to feature the home with words such as: **victorian, backyard, garden, deck, private entrance** and **yard**. In contrast, listing descriptions with all other ratings focus more on the amenities of the property with words like: **bedding, wifi, linens, screen tv, email,** and **fitness**. While it is important to list amenities, these same words could also be used to describe a hotel, and does little to describe the individuality of a home. 

## 5.3 Lexical categories

In Figure 3, the most frequent topic in listing descriptions with high star ratings is **royalty**. This theme is repeated 136 times/25,000 terms versus only 77 times/25,000 terms for all other ratings. In other words, listing descriptions with high star ratings use words associated with the theme royalty almost twice as much as all other ratings! Some of these words include, magnificent, queen, elegant, supreme and distinguished. 

But this is slightly misleading when taken out of context. While some of the listing descriptions may use words that grammatically match the theme royalty, the majority of these words are **queen** and **king**. They are used instead to describe the size of the bed. Still, these terms are used nearly twice as much, and it can be argued that listing descriptions including more mentions of luxuriously-sized beds, are more likely to attract a higher rating among guests. 
    

The most frequent topic in listing descriptions with all other star ratings is **stealing**. Curiously, **property** appears to be the seed term for this lexical category. Property is mentioned 108 times/25,000 terms versus 26 times/25,000 terms for high star ratings. Examining the excerpts show that property is most commonly used in listing descriptions by other home-sharing companies like Sonder & Zeus. These companies refer to their living spaces as properties, rather than homes. This is fascinating because it reveals distinct linguistic disparities. Private homeowners don't view their homes as property, but a **home**, with all of its associated homey connotations. On the other hand, companies view these same homes as **property**, another asset in their portfolio. 

Interestingly, we see less **property** mentions in listing descriptions with high star ratings, which goes in hand with the above analysis that private homes enjoy higher ratings than properties owned by companies.
    
    
When looking at some of the other terms on the list such as negative_emotion, violence and nervousness, keep in mind these categories have been taken out of context. For example, violence was inferred from: 
    
*the city's **beating** heart, **kick** back in the suite on its own private floor, **hit** up The Mill for some liquid energy*

## 5.4 Word similarities

The plot in Figure 4 reveals two semi-distinct shapes, a central cluster (all other ratings) and an outer ring structure (high star ratings).


Geography is clustered in the upper right corner, with neighborhoods and outdoor attractions, like the pier and Alcatraz. Transportation is captured to the right of the plot, with words like Lyft, Muni, and Bart. It is interesting to see how similar words are clustered, and then whether these words belong to highly rated descriptions or all others. For example, one of my favorite word similarities is how closely **pet** (associated with all other ratings) and **kid** (associated with high star rating) are clustered together. Homes more accommodating towards kids, are more favorable, while pets are rarely allowed.

## 5.5 BERT text classifier

The model achieved 81.3% accuracy on a validation set of 1,780 listing descriptions. 

When I tested the BERT classifier on text from a listing description, it claimed the listing would fall into the 3-4 star category. The concern here though is that each category step is quite wide. A home receiving 3 stars, and a home receiving 4 stars would both belong in the same category but obviously, there is a significant gap between these two. A 3-star home would be much less preferable than a 4-star home. Currently, the rating system does not allow me to capture these finer in-between scoring steps, and I recommend obtaining a score of at least 4-5 stars to be confident in your listing description. 

There is also an issue of score distributions. It is unlikely for low-rated homes to remain on the platform for long, and as a result, nearly all of the homes on Airbnb have 3 or more stars. This creates an even more difficult task for the classifier. Homes with a weak listing description may receive the same star rating as a home with a much stronger, and stylistic description. As long as the guest feels like their stay was fine, regardless of the listing description, the home will receive an average or above-average rating. As a result, the model must learn that sometimes even weak listing descriptions will yield a decent rating. 

This may lead you to wonder then, why should hosts even bother with a listing description? The answer is because the listing description plays an important role in the probability of guests booking a home. When all the ratings are fairly similar, we begin to compare other factors, like how stylistic and unique a listing description is. Stronger listing descriptions increases the probability of a guest booking a home. Weaker listing descriptions may only have a history of three or four guests staying there, even though they have an above-average rating. Therefore, it is still important to write a great listing description.

## 5.6 Recommendations

1. Focus on describing what makes your home unique, instead of amenities. Does it have a garden for guests to sit out in and enjoy? Or perhaps a deck with a gorgeous view of the city? These types of descriptions are more apt to earn a high star rating.


2. Describe your home with terms that evoke feelings of warmth and home, rather than detailing a list of amenities, and describing your space in an impersonal style.

## 5.7 Next steps

A couple projects that could build off of this body of work include:

1. Creating a text generator in the Airbnb Plus editorial style, to make it even easier for hosts to quickly create a top-rated listing description.


2. Training a classifier on a more stratified rating system to help hosts diagnose their listing descriptions in greater detail.


3. These listing descriptions are unique to San Francisco, and it would be interesting to compare it with listing descriptions from another city, to observe regional discrepancies in what makes a home attractive. This could also be a fascinating study in how other cultures perceive their homes, as reflected in their text.